In [1]:
import requests

# define the API endpoint to get transaction data
api_endpoint = "https://blockchain.info/rawaddr/"

# specify the Bitcoin address we want to analyze
address = "1HLoD9E4SDFFPDiYfNYnkBLQ85Y51J3Zb1"

# make a request to the API to get transaction data for the address
response = requests.get(api_endpoint + address)

# parse the JSON data into a Python dictionary
transaction_data = response.json()

In [8]:
import pandas as pd

# convert the transaction data into a Pandas DataFrame
df = pd.DataFrame(transaction_data["txs"])

# clean the data by removing unnecessary columns
df = df[["time", "inputs", "out"]]

# convert the timestamps to datetime objects
df["time"] = pd.to_datetime(df["time"], unit="s")

# create a new column to track the total transaction volume
df["total_output_value"] = df["out"].apply(lambda x: sum([o["value"] for o in x]))

# remove any rows where the total transaction volume is 0
df = df[df["total_output_value"] > 0]

In [15]:
# create a new column to track the hour of the day when the transaction occurred
df["hour_of_day"] = df["time"].apply(lambda x: x.hour)

# create a new column to track the day of the week when the transaction occurred
df["day_of_week"] = df["time"].apply(lambda x: x.weekday())

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# define our features and target variable
X = df[["hour_of_day", "day_of_week"]]
y = df["total_output_value"]

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train a linear regression model on the training data
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions on the testing data and evaluate the model's performance
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 2.676732681694596e+18


In [18]:
# use the trained model to make predictions on new data
new_data = pd.DataFrame({
    "hour_of_day": [10],
    "day_of_week": [2]
})
prediction = model.predict(new_data)[0]
print("Predicted Total Output Value:", prediction)

Predicted Total Output Value: 166001361.77263957
